# Đề thi mẫu chứng chỉ TensorFlow - Dạng bài 03

```
ProtonX - TensorFlow Class
```
Giới thiệu về chứng chỉ - https://www.tensorflow.org/certificate

> Chú ý: đọc kỹ các nội dung trong Handle Notebook trong liên kết trên.

---

### Hướng dẫn làm bài

Đọc kỹ File `Tensorflow Exam - Tips` trên hệ thống Học tập.

*Một số yêu cầu:
- Sử dụng TensorFlow 2.x trở lên.

Phân loại ảnh thực tế:
- Một số loại dataset thường gặp trong đề thi: Chó - mèo, Ngựa - người, Dao - Búa - Kéo.
- Có thể đề yêu cầu load data từ tf.dataset. Đề bài có thể yêu cầu resize ảnh về kích thước nào đó (ví dụ 224x224x3) và re-scale input về khoảng [-1, 1]. Thực hiện resize và rescale ảnh trong hàm map:
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map

- Lưu ý 1: Có thể sử dụng Transfer learning - ưu tiên MobileNet để Train mô hình có độ chính xác cao hơn.
- Lưu ý 2: Phân loại nhị phân nhưng đề bài có thể yêu cầu lớp cuối cùng của model phải sử dụng hàm softmax(thay vì sigmoid), vì vậy cần chọn làm loss phù hợp với hàm softmax.
- flow_from_directory + generator
  + Chó mèo: https://youtu.be/kShvYHDXvvg
  + Ngựa người: https://youtu.be/ee9tF9xEf04
  + Dao búa kéo: https://www.tensorflow.org/datasets/catalog/rock_paper_scissors
  + Cách resize ảnh, rescale input và transfer learning có thể tham khảo: https://www.tensorflow.org/tutorials/images/transfer_learning
https://youtu.be/Y-4KLFt_c6Y

- Bài sử dụng transfer learning có thể có kết quả khi submit là 2/5, 3/5, …. Vì vậy nên submit nhiều lần. Nếu đạt từ 4/5 trở lên thì có thể làm bài khác (áp dụng với tất cả các bài)

- Chạy lại toàn bộ File này trước khi ấn nộp bài.

### Đề thi mẫu bài 03

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

print(tf.__version__)

2.15.0


In [2]:
# Các bạn có thể tải dữ liệu bằng đường liên kết này
!wget https://storage.googleapis.com/download.tensorflow.org/data/rps.zip -O /tmp/rockpaperscissors.zip

--2024-02-08 14:53:29--  https://storage.googleapis.com/download.tensorflow.org/data/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.207, 108.177.98.207, 74.125.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 191.38M   229MB/s    in 0.8s    

2024-02-08 14:53:30 (229 MB/s) - ‘/tmp/rockpaperscissors.zip’ saved [200682221/200682221]



In [3]:
# Hoặc tải dữ liệu trong TensorFlow Dataset
DATASET_NAME = 'rock_paper_scissors'
IMG_SIZE = 300
INP_SHAPE = (IMG_SIZE,IMG_SIZE,3)
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

(ds_train, ds_test), ds_info = tfds.load(
    DATASET_NAME,
    split = ['train', 'test'],
    shuffle_files = True,
    as_supervised = True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/2520 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteYUUXM8/rock_paper_scissors-train.tfrec…

Generating test examples...:   0%|          | 0/372 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteYUUXM8/rock_paper_scissors-test.tfreco…

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


In [4]:
resize_and_rescale = tf.keras.models.Sequential([
    tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE),
    tf.keras.layers.Rescaling(1./255),
])
data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.RandomFlip(mode = 'horizontal_and_vertical'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])
def prepare(ds, shuffle = False, augment = False):
  ds = ds.map(lambda x,y: (resize_and_rescale(x),y),num_parallel_calls = AUTOTUNE)
  if shuffle:
    ds = ds.shuffle(1000)
  ds = ds.batch(batch_size)

  if augment:
    ds = ds.map(lambda x,y: (data_augmentation(x,training=True), y), num_parallel_calls = AUTOTUNE)
  return ds.prefetch(buffer_size = AUTOTUNE)

In [5]:
ds_train = prepare(ds_train, shuffle= True, augment = True)
ds_test = prepare(ds_test)

In [6]:
ds_train

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [7]:
ds_test

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (INP_SHAPE)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation = 'softmax')


])

In [9]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

In [10]:
model.fit(ds_train, batch_size = 32, verbose = 1, epochs = 100, validation_data = ds_test)

Epoch 1/100
79/79 [==============================] - 58s 597ms/step - loss: 1.1390 - acc: 0.3440 - val_loss: 1.0975 - val_acc: 0.3333
Epoch 2/100
79/79 [==============================] - 49s 581ms/step - loss: 1.0762 - acc: 0.4008 - val_loss: 1.0230 - val_acc: 0.3898
Epoch 3/100
79/79 [==============================] - 49s 587ms/step - loss: 0.9560 - acc: 0.5345 - val_loss: 0.5845 - val_acc: 0.9032
Epoch 4/100
79/79 [==============================] - 49s 575ms/step - loss: 0.6808 - acc: 0.7119 - val_loss: 0.4514 - val_acc: 0.8898
Epoch 5/100
79/79 [==============================] - 49s 579ms/step - loss: 0.5047 - acc: 0.7905 - val_loss: 0.3872 - val_acc: 0.8925
Epoch 6/100
79/79 [==============================] - 49s 580ms/step - loss: 0.4475 - acc: 0.8107 - val_loss: 0.4173 - val_acc: 0.7930
Epoch 7/100
79/79 [==============================] - 48s 576ms/step - loss: 0.4062 - acc: 0.8421 - val_loss: 0.3536 - val_acc: 0.8172
Epoch 8/100
79/79 [==============================] - 48s 584ms

In [11]:
# Lưu Model và nộp
model.save("mymodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
